In [ ]:
#Install the library for reading the NC file
!pip install netCDF4

# Importing Necessary Libararies
import numpy as np
import pandas as pd
from netCDF4 import Dataset
from scipy.spatial import distance
from sklearn import preprocessing
from scipy.spatial import distance
import os

In [ ]:
file_path = '/content/drive/MyDrive/Project/Data/Weather_Data/2024/fg_ens_mean_0.1deg_reg_2011-2024_v30.0e.nc'
weather_feature = Dataset(file_path, mode='r')
weather_name = (file_path.split('/'))[-1].split('_')[0]

#To find the closest european station for each station in the project, we need to find the min value for both latitude and longitude
min_latitude = round(float(min(weather_feature.variables['latitude'])),2)
min_longitude = round(float(min(weather_feature.variables['longitude'])),2)
print(min_latitude,min_longitude)

latitude= pd.Series(weather_feature.variables['latitude'][:].data)
longitude= pd.Series(weather_feature.variables['longitude'][:].data)
EuropeLocation = []
for lat in latitude:
  for lon in longitude:
    EuropeLocation.append((lat,lon,))

EuropeLocation

In [ ]:
# Reading 'postcode_of_station' dataset in order to find the latitude and longitude of stations defined in the project (available by airquality data)
Station_locations = pd.read_csv('/content/drive/MyDrive/Project/Data/AirQuality/postcode_of_station.csv')
Station_locations['latitude'] = Station_locations['latitude'].round(decimals = 6)
Station_locations['longitude'] = Station_locations['longitude'].round(decimals = 6)

# To work on spcific country, just modify the param called country.
# Spain = 1 , Italy = 2 , Porugal = 3
coutry_dict = {1:'Spain',2:'Italy',3:'Portugal'}
for j in range(1,3):
  country_stations = Station_locations [ Station_locations.apply(lambda row : row['country'] == j, axis = 1) ]
  country_locations = country_stations[ ['id_postcode_of_station','latitude','longitude'] ]
  Locations = list(zip(country_locations['latitude'], country_locations['longitude']))

  result = []
  for loc in Locations:
    temp = []
    for Eloc in EuropeLocation:
      temp.append([distance.euclidean(loc,Eloc),Eloc])
    closest_distance = min(temp)
    result.append((loc,closest_distance))

  #Making DataFrame for downloading the extracted data and Saving it in CSV format
  Findings_distances = pd.DataFrame(result, columns= ['Station Location', 'Closest Distance of EuropeLocation with Station Location'])

  #Preprocess the distance dataframe
  empty_df = pd.DataFrame()
  empty_df['Station Latitude'] = Findings_distances['Station Location'].apply(lambda row: row[0])
  empty_df['Station Latitude'] = empty_df['Station Latitude'].round(decimals = 6)

  empty_df['Station Longitude'] = Findings_distances['Station Location'].apply(lambda row: row[1])
  empty_df['Station Longitude'] = empty_df['Station Longitude'].round(decimals = 6)

  empty_df['E-OBS Latitude'] = Findings_distances['Closest Distance of EuropeLocation with Station Location'].apply(lambda row: row[1][0])
  empty_df['E-OBS Longitude'] = Findings_distances['Closest Distance of EuropeLocation with Station Location'].apply(lambda row: row[1][1])

  empty_df['Index Lat'] = ((empty_df['E-OBS Latitude'].round(2)-min_latitude)*10).round(0).astype(int)
  empty_df['Index Long'] = ((empty_df['E-OBS Longitude']-min_longitude)*10).round(0).astype(int)

  empty_df['Distance'] = Findings_distances['Closest Distance of EuropeLocation with Station Location'].apply(lambda row: row[0])

  ClosestDistanceByCountry = pd.merge(empty_df,Station_locations,how = 'left', left_on=['Station Latitude','Station Longitude'], right_on =['latitude','longitude'])
  ClosestDistanceByCountry = ClosestDistanceByCountry[ClosestDistanceByCountry.columns[:10]]

  ClosestDistanceByCountry.to_csv(f'ClosestDistanceBy{coutry_dict[j]}.csv', index = False)

In [ ]:
# Read weather data for each coutry seperately
countries = ['Spain','Italy','Portugal']
closest_folder_path = '/content/drive/MyDrive/Project/Data/Weather_Data/2024/'
list_path = os.listdir(closest_folder_path)
for i in range(len(list_path)):
  path = closest_folder_path+list_path[i]
  weather_feature = Dataset(path, mode='r')
  weather_name = (path.split('/'))[-1].split('_')[0]
  for c in countries:
    if c == 'Spain':
      Spain = pd.read_csv('/content/ClosestDistanceBySpain.csv')
      data_tm = {}
      for loc in range(len(Spain)):
        lat = Spain['Index Lat'][loc]
        lon = Spain['Index Long'][loc]
        temp = []
        for tm in range(len(weather_feature.variables[weather_name][4748:])): #For 2024 the index started from 4748
          temp.append(weather_feature.variables[weather_name][tm].data[lat,lon])
        data_tm[loc]= temp
        print(c,weather_name)
        print(loc, temp)
      dataset = pd.DataFrame(data_tm)
      dataset = dataset.T
      dataset.to_csv(f'{weather_name}_weather_Bigdata_{c}.csv', index = False)
      continue
    elif countries == 'Italy':
      Italy = pd.read_csv('/content/ClosestDistanceByItaly.csv')
      data_tm = {}
      for loc in range(len(Italy)):
        lat = Italy['Index Lat'][loc]
        lon = Italy['Index Long'][loc]
        temp = []
        for tm in range(len(weather_feature.variables[weather_name][4748:])): #For 2024 the index started from 4748
          temp.append(weather_feature.variables[weather_name][tm].data[lat,lon])
        data_tm[loc]= temp
        print(c,weather_name)
        print(loc, temp)
      dataset = pd.DataFrame(data_tm)
      dataset = dataset.T
      dataset.to_csv(f'{weather_name}_weather_Bigdata_{c}.csv', index = False)
      continue
    else:
      Portugal = pd.read_csv('/content/ClosestDistanceByPortugal.csv')
      data_tm = {}
      for loc in range(len(Portugal)):
        lat = Portugal['Index Lat'][loc]
        lon = Portugal['Index Long'][loc]
        temp = []
        for tm in range(len(weather_feature.variables[weather_name][4748:])): #For 2024 the index started from 4748
          temp.append(weather_feature.variables[weather_name][tm].data[lat,lon])
        data_tm[loc]= temp
        print(c,weather_name)
        print(loc, temp)
      dataset = pd.DataFrame(data_tm)
      dataset = dataset.T
      dataset['StationLatitude'] = Portugal['Station Latitude']
      dataset['StationLongitude'] = Portugal['Station Longitude']
      dataset.to_csv(f'{weather_name}_weather_Bigdata_{c}.csv', index = False)
      continue

In [ ]:
# Integrate and preprocess data
result = []
folder_path = '/content/drive/MyDrive/Project/Data/Weather_Data/2024/'
list_path = os.listdir(folder_path)
for i in range(len(list_path)):

  country_path = os.listdir(folder_path+list_path[i])
  empty_df = pd.DataFrame(columns = ['id','postcode','id_postcode_of_station','measurement','date','value'])

  for j in range(len(country_path)):

    path = folder_path+list_path[i]+'/'+country_path[j]
    data = pd.read_csv(path)
    splited_path = (path.split('/'))[-1].split('_')

    if list_path[i] == 'Spain':

      data['id_postcode_of_station'] = Spain['id_postcode_of_station']
      data['StationLatitude'] = Spain['Station Latitude']
      data['StationLongitude'] = Spain['Station Longitude']

    elif list_path[i] == 'Italy':

      data['id_postcode_of_station'] = Italy['id_postcode_of_station']
      data['StationLatitude'] = Italy['Station Latitude']
      data['StationLongitude'] = Italy['Station Longitude']

    else:

      data['id_postcode_of_station'] = Portugal['id_postcode_of_station']
      data['StationLatitude'] = Portugal['Station Latitude']
      data['StationLongitude'] = Portugal['Station Longitude']

    for k in range(data.shape[0]):

        row = data.iloc[k]
        temp = row[4:].reset_index() # 3: means value params
        temp.columns = ['date','value']
        temp['id_postcode_of_station'] = row.loc['id_postcode_of_station']
        temp['measurement'] = splited_path[0]
        temp['date'] = pd.date_range(start='1/1/2024', end='30/06/2024')
        empty_df = empty_df._append(temp, ignore_index=True)

  if list_path[i] == 'Spain':

    empty_df['id'] = range(1,empty_df.shape[0]+1)
    empty_df['id_postcode_of_station'] = empty_df['id_postcode_of_station'].astype(int)
    result.append(empty_df['id'].iloc[-1])

  elif list_path[i] == 'Italy':

    empty_df['id'] = range((result[0]+1),(result[0] + len(empty_df)+1))
    empty_df['id_postcode_of_station'] = empty_df['id_postcode_of_station'].astype(int)
    result.append(empty_df['id'].iloc[-1])

  else:

    empty_df['id'] = range((result[1]+1),(result[1] + len(empty_df)+1))
    empty_df['id_postcode_of_station'] = empty_df['id_postcode_of_station'].astype(int)

  empty_df['measurement'] = empty_df['measurement'].map({'fg' : 8,
                                                         'hu' : 9,
                                                         'pp' : 10,
                                                         'qq' : 11,
                                                         'rr' : 12,
                                                         'tg' : 13,
                                                         'tn' : 14,
                                                         'tx' : 15})
  empty_df.rename(columns={"value": "measurement",
                           "id_postcode_of_station": "postcode",
                           "date": "timestamp",
                           "measurement": "param" }, inplace = True)

  empty_df.to_csv(f'Weather_Final_Data_{list_path[i]}.csv', index = False)
